# 데이터 증폭을 배열로 설정할 것. 

In [6]:
# 데이터 증강 배수 설정 
heat_road_multiple = 5

In [7]:
import pandas as pd 
import numpy as np
from tqdm import tqdm

In [8]:
url = './data/(최종)_서울열선_광진도로.csv'
encoding = "UTF-8"

In [9]:
df = pd.read_csv(url, encoding = encoding)

In [10]:
len(df)

7212

In [11]:
df['원본여부'] = 1

In [12]:
df.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모', '행정구역', '고도의_차이', '경사각',
       '최근접_시설들_거리', '최근접_시설의_평균거리', '원본여부'],
      dtype='object')

In [13]:
try :
    delete_columns = 'Unnamed: 0'
    df = df.drop(columns = delete_columns)
    print(f"{delete_columns} 삭제 완료")
except KeyError as e :
    print(e)

"['Unnamed: 0'] not found in axis"


In [14]:
len(df)

7212

In [15]:
const_heat_data = df[df['열선'] == 1]
heat_data = df[df['열선'] == 1]
road_data = df[df['열선'] == 0]

In [16]:
print(len(heat_data))
print(len(road_data))

1051
6161


In [17]:
heat_data.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '행정동', '도로폭', '도로규모', '행정구역', '고도의_차이', '경사각',
       '최근접_시설들_거리', '최근접_시설의_평균거리', '원본여부'],
      dtype='object')

## 열선 데이터를 늘리기 시작

In [18]:
for road_idx in range(heat_road_multiple):
    temp_data = const_heat_data.copy()

    for data_idx in tqdm(range(len(temp_data))):
        temp_data.loc[data_idx, '원본여부'] = 0  
        
        if "," in temp_data.loc[data_idx, '도로명']:  
            temp_1 = temp_data.loc[data_idx, '도로명']
            temp_2 = temp_1.split(',')[0]   
            temp_3 = temp_2.replace("[\'", "")
            temp_4 = temp_3.replace("\'", '')
            temp_data.loc[data_idx, '도로명'] = temp_4

        temp_data.loc[data_idx, '경사각'] = abs(temp_data.loc[data_idx, '경사각'] + np.random.uniform(-7, 7))

        if temp_data.loc[data_idx, '최근접_시설들_거리']:
            distances = temp_data.loc[data_idx, '최근접_시설들_거리'].split(",")  
            distances = [float(d.strip()) + np.random.uniform(-5, 5) for d in distances]
            temp_data.loc[data_idx, '최근접_시설들_거리'] = ", ".join(map(str, distances))
            temp_data.loc[data_idx, '최근접_시설의_평균거리'] = np.mean(distances)

    heat_data = pd.concat([heat_data, temp_data], axis=0, ignore_index=True)

100%|█████████████████████████████████████| 1051/1051 [00:00<00:00, 5520.42it/s]


In [19]:
len(heat_data)

6306

In [20]:
total_data = pd.concat([heat_data, road_data], axis=0, ignore_index=True)

In [21]:
print(total_data['열선'].value_counts())
print("------------------------")
print(total_data['원본여부'].value_counts())

열선
1    6306
0    6161
Name: count, dtype: int64
------------------------
원본여부
1    7212
0    5255
Name: count, dtype: int64


In [22]:
total_data.isnull().sum()

도로명              0
열선               0
도로 종류            0
시작점_위도           0
시작점_경도           0
종료점_위도           0
종료점_경도           0
중앙점_위도           0
중앙점_경도           0
도로_길이            0
행정동              0
도로폭             79
도로규모            79
행정구역             0
고도의_차이           0
경사각              0
최근접_시설들_거리       0
최근접_시설의_평균거리     0
원본여부             0
dtype: int64

In [23]:
try :
    save_url = "./data/(증폭)_서울열선_광진도로.csv"
    total_data.to_csv(save_url, encoding="UTF-8", index=False)
    print(f"{save_url} 저장 완료.")
except OSError as e :
    print(e)

./data/(증폭)_서울열선_광진도로.csv 저장 완료.
